https://judgments.ecourts.gov.in/pdfsearch/

In [1]:
#########################################
## Import Selenium and other libraries ##
#########################################

import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

In [2]:
#######################################
## Import captcha breaking libraries ##
#######################################

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import matplotlib.pyplot as plt
import cv2

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')

/Users/shubhamgulati/.pyenv/versions/3.10.6/envs/mindcase-scraping/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
###############################
## Loading the Chrome Driver ##
###############################

# Specify the path to the ChromeDriver executable
PATH = r"/Users/shubhamgulati/Downloads/chromedriver-mac-arm64/chromedriver"

# Set the ChromeDriver executable path using the Service class
service = webdriver.chrome.service.Service(PATH)

In [ ]:
#################################
## Captcha Extraction Function ##
#################################

def extract_captcha():
    # Load the CAPTCHA image
    image = cv2.imread("captchaa.png")

    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # plt.imshow(image)
    # plt.show()

    print(generated_text)

    return generated_text

In [ ]:
#################################
## Initiate WebDriver Instance ##
#################################

# Create a Chrome WebDriver instance with the service
driver = webdriver.Chrome(service=service)

# Create a WebDriverWait instance
wait = WebDriverWait(driver, 10)

In [ ]:
###########################################
## Open new link and break first captcha ##
###########################################

def open_new_link():
    # Navigate to the URL
    driver.get('https://judgments.ecourts.gov.in/pdfsearch/')

    print('Link opened successfully') 

    # Get search field using the 'search_text' id
    # search_field = driver.find_element(By.ID, 'search_text')
    # search_field.send_keys('iim')
    # print('Search text entered')

    # Get captcha image using the 'captcha_image' id
    captcha_element = driver.find_element(By.ID, 'captcha_image')

    print('Captcha image found')

    # Save a screenshot of the CAPTCHA image
    with open("captchaa.png", "wb") as captcha_file:
        captcha_file.write(captcha_element.screenshot_as_png)

    print('Captcha image saved')

    # Extract the CAPTCHA text from the image
    captcha_text = extract_captcha()

    print('Captcha text extracted')

    # Enter captcha text in the 'captcha' text field
    captcha_text_element = driver.find_element(By.ID, 'captcha')
    captcha_text_element.send_keys(captcha_text)

    print('Captcha text entered')

    # Click on the 'main_search' button
    search_button = driver.find_element(By.ID, 'main_search')
    search_button.click()

    print('Search button clicked')

    sleep(0.5)

    # Wait for the page to load
    wait.until(EC.visibility_of_element_located((By.ID, 'example_pdf_length')))

    # wait for results to load
    wait.until(EC.invisibility_of_element_located((By.ID, 'loadMe.show')))

    print('Results page loaded')

    # open select element by name 'example_pdf_length'
    sel = Select(driver.find_element(By.NAME, 'example_pdf_length'))

    # select value 1000
    sel.select_by_value('1000')

    sleep(0.5)

    # wait for results to load
    wait.until(EC.invisibility_of_element_located((By.ID, 'loadMe.show')))

    print('Results showing 1000 entries')

open_new_link()

Link opened successfully
Captcha image found
Captcha image saved
502277
Captcha text extracted
Captcha text entered
Search button clicked
Results page loaded
Results showing 1000 entries


In [ ]:
############################################
## Download the first pdf (Test Function) ##
############################################
## DON'T CALL THIS FUNCTION IN THE SCRIPT ##
############################################

def download_pdf_testing():
    # Click button with id 'link_0' to open the first result
    link_0 = driver.find_element(By.ID, 'link_0')
    link_0.click()

    print('Link_0 clicked')

    # wait for the new tab to open
    wait.until(EC.number_of_windows_to_be(2))

    # switch to the new tab
    driver.switch_to.window(driver.window_handles[1])

    # copy the link of the new tab
    link = driver.current_url
    print(link)

    # download the pdf
    destination = '1.pdf'
    response = requests.get(link)
    with open(destination, 'wb') as output_file:
        output_file.write(response.content)

    # close the new tab
    driver.close()

    # wait to close the link
    wait.until(EC.number_of_windows_to_be(1))

    print('New tab closed')

    # switch back to the original tab
    driver.switch_to.window(driver.window_handles[0])

    print('Link opened successfully')

In [ ]:
#############################################
## Function to Download PDF given by index ##
#############################################

# index: index of the pdf in the results (0 to total_results)
# id: id of the pdf on the page (0 to number_of_entries_on_the_page)
def download_pdf(index, id):
    # Click button with id 'link_0' to open the first result
    link = driver.find_element(By.ID, 'link_' + str(id))
    driver.execute_script("arguments[0].click();", link)
    # driver.execute_script("arguments[0].scrollIntoView(true);", link)
    # wait.until(EC.element_to_be_clickable((By.ID, 'link_' + str(id))))
    # link.click()

    print('Link_' + str(id) + ' clicked')

    # wait for the new tab to open
    wait.until(EC.number_of_windows_to_be(2))

    # switch to the new tab
    driver.switch_to.window(driver.window_handles[1])

    # copy the link of the new tab
    url = driver.current_url
    print(url)

    # download the pdf
    destination = 'allpdfs/' + str(index) + '.pdf'
    while True:
        print('Downloading the pdf')
        try:
            response = requests.get(url)
            with open(destination, 'wb') as output_file:
                output_file.write(response.content)
            break
        except:
            sleep(3)
            continue

    print('Downloaded ' + str(index) + '.pdf')

    # close the new tab
    driver.close()

    # wait to close the link
    wait.until(EC.number_of_windows_to_be(1))

    print('New tab closed')

    # switch back to the original tab
    driver.switch_to.window(driver.window_handles[0])

    print('Link opened successfully')

    sleep(0.5)

In [ ]:
#####################################
## Function to go to the next page ##
#####################################

def next_page():
    # click button with id 'example_pdf_next'
    next_page = driver.find_element(By.ID, 'example_pdf_next')
    driver.execute_script("arguments[0].click();", next_page)
    # driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
    # wait.until(EC.visibility_of_element_located((By.ID, 'example_pdf_next')))
    # next_page.click()
    # wait while #loadMe has class 'hide show'
    wait.until(EC.invisibility_of_element_located((By.ID, 'loadMe.show')))
    sleep(2)
    print('Next page opened successfully')


In [ ]:
########################################
## Main function to download all pdfs ##
########################################

def download_all_pdfs(start):
    for i in range(start, 1000):
        if i % 20 == 0:
            while True:
                try:
                    open_new_link()
                    break
                except:
                    continue
        download_pdf(i, i)

In [ ]:
####################################################
## Execute the main function to download all pdfs ##
####################################################

download_all_pdfs(680)

Link_618 clicked
https://judgments.ecourts.gov.in//judgments_lib/tmp/21958308a135ee944724dfc8b2b79703c32ccb14bd4b55be01ccfba9d349cac31699635617.pdf
Downloaded 618.pdf
New tab closed
Link opened successfully
Link_619 clicked
https://judgments.ecourts.gov.in//judgments_lib/tmp/138bc6375070765dff6592a928bca3c27fb00e84466a8880d7bb12bee55c96291699635627.pdf
Downloaded 619.pdf
New tab closed
Link opened successfully
Link opened successfully
Captcha image found
Captcha image saved
028672
Captcha text extracted
Captcha text entered
Search button clicked
Link opened successfully
Captcha image found
Captcha image saved
634546
Captcha text extracted
Captcha text entered
Search button clicked
Results page loaded
Results showing 1000 entries
Link_620 clicked
https://judgments.ecourts.gov.in//judgments_lib/tmp/d7128d1fa756dc66368c9c386055aba9bcc1621d35f23be8fb257679c75296491699635657.pdf
Downloaded 620.pdf
New tab closed
Link opened successfully
Link_621 clicked
https://judgments.ecourts.gov.in//jud

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
open_new_link()

Link opened successfully
Captcha image found
Captcha image saved
133903
Captcha text extracted
Captcha text entered
Search button clicked
Results page loaded
Results showing 1000 entries
